In [27]:
import requests

In [28]:
response = requests.get('https://api.github.com/events')#, verify=False)
print(response.status_code)
print(response.url)
print(response.text)
print(response.json())

200
https://api.github.com/events
[{"id":"38336700615","type":"CreateEvent","actor":{"id":118344674,"login":"github-merge-queue[bot]","display_login":"github-merge-queue","gravatar_id":"","url":"https://api.github.com/users/github-merge-queue[bot]","avatar_url":"https://avatars.githubusercontent.com/u/118344674?"},"repo":{"id":308027791,"name":"smithy-lang/smithy-rs","url":"https://api.github.com/repos/smithy-lang/smithy-rs"},"payload":{"ref":"gh-readonly-queue/main/pr-3618-e3a9c8c30c42b859e76af04582e310c18964879f","ref_type":"branch","master_branch":"main","description":"Code generation for the AWS SDK for Rust, as well as server and generic smithy client generation.","pusher_type":"user"},"public":true,"created_at":"2024-05-13T20:08:58Z","org":{"id":133827778,"login":"smithy-lang","gravatar_id":"","url":"https://api.github.com/orgs/smithy-lang","avatar_url":"https://avatars.githubusercontent.com/u/133827778?"}},{"id":"38336700599","type":"PullRequestEvent","actor":{"id":132092648,"lo

## Projeto

In [29]:
import requests

#### Autenticação

Solicitações autenticadas têm um limite de taxa mais alto. Quando um usuário faz uma solicitação autenticada, ele fornece credenciais que comprovam sua identidade, o que permite que a API confie nele e lhe conceda acesso a recursos e funcionalidades adicionais.

Além disso, a maioria das APIs estabelece limites para o número de solicitações que um usuário pode fazer em um determinado período de tempo, conhecido como 'limite de taxa'. Quando um usuário faz solicitações autenticadas, a API geralmente permite que ele faça mais solicitações em um determinado período de tempo, devido à maior confiança e credibilidade que a autenticação fornece.

In [86]:
access_token = ' '
headers = {'Authorization': 'Bearer ' + access_token,
           'X-GitHub-Api-Version': '2022-11-28'}

Ao fazer uma requisição para a API REST do GitHub usando a biblioteca Requests do Python, você pode especificar a versão da API que deseja acessar através da adição do header. 

````
headers = {'X-GitHub-Api-Version': '2022-11-28'}
````

#### url da API

In [31]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # username de quem vamos extrair os dados
url = f'{api_base_url}/users/{owner}/repos'
url

'https://api.github.com/users/amzn/repos'

#### Paginação

Extrair os dados de todos os repositórios do usuário de GitHub da Amazon.

In [32]:
repos_list = []
# Percorre as páginas do repositório da amazon
for page_num in range(1, 6):
    # Tratamento de erro
    try:
        url_page = f'{url}?page={page_num}'
        response = requests.get(url_page, headers=headers)#,verify=False)
        response.status_code
        repos_list.append(response.json())
    # caso ocorra algum erro
    except:
        repos_list.append(None)

#### Nome dos repositórios

In [33]:
# Acessar o repositorio 3 da pagina 1
repos_list[0][2]['name']

'ads-pao-amznjs-gtm-template'

In [34]:
respo_name = []
for page in repos_list:
    for repo in page:
        respo_name.append(repo['name'])
len(respo_name)

150

### Nome das linguagens

In [35]:
repos_list[0][2]['language']

'Smarty'

In [40]:
respo_language = []
for page in repos_list:
    for repo in page:
        respo_language.append(repo['language'])
respo_language[:4]

[None, None, 'Smarty', 'JavaScript']

### Criando a base de dados

In [38]:
import pandas as pd

In [42]:
dataset = pd.DataFrame()
dataset['repository_name'] = respo_name
dataset['language'] = respo_language
dataset.head(4)

,repository_name,language
0,.github,None
1,ads-advanced-tools-docs,None
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript


### Salvando dados

In [72]:
path = r"C:\Users\mathe\OneDrive\Profissional\Projetos\Engenharia de dados\pipeline-languages-companies\src\data\amazon.csv"
dataset.to_csv(path)

### Salvando em um repositório


In [85]:
api_base_url = 'https://api.github.com'
url = f'{api_base_url}/user/repos'
url

'https://api.github.com/user/repos'

### criação do repositorio

In [47]:
data = {
    'name': 'pipeline-languages-companies',
    'description': 'Pipeline de dados: extraindo, transformando e carregando dados acerca das linguagens de programação utilizadas pela a Amazon',
    'private': False
}

In [49]:
reponse = requests.post(url, json = data,headers=headers)
reponse.status_code

422

### conversão do arquivo para Base64

content é o parâmetro que utilizaremos para passar o arquivo no momento de fazer o upload dele para nosso repositório no GitHub. A descrição desse parâmetro informa que o conteúdo do novo arquivo deve estar codificado em Base64. 

A codificação em Base64 é uma técnica que permite transformar arquivos em uma sequência de caracteres que pode ser transmitidas facilmente pela internet sem perder nenhuma informação.

In [70]:
import base64

In [73]:
with open(path, 'rb') as file:
    file_content = file.read()

encoded_content = base64.b64encode(file_content)

### Carregando os dados

In [114]:
api_base_url = 'https://api.github.com'
username = 'matheussooares'
repo = 'pipeline-languages-companies'
path = 'src/data/amazon.csv'
url = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'
url

'https://api.github.com/repos/matheussooares/pipeline-languages-companies/contents/src/data/amazonn.csv'

In [115]:
data = {
    'message': 'add file csv',
    'content': encoded_content.decode('utf-8')
}

In [116]:

reponse = requests.put(url, json = data,headers=headers)
reponse.status_code

201